In [ ]:
# def save_scene_images():
#     for idx, row in tqdm(df.iterrows(), total=len(df)):
#         img_path = os.path.join(findingemo_dir, row['image_path'])
#         boxes = df_boxes[df_boxes['index'] == idx][['x1', 'y1', 'x2', 'y2']].values.tolist()
#         blurred_image = blur_faces(img_path, boxes)
#         blurred_image = cv2.resize(blurred_image, (224, 224))
#         save_path = os.path.join(findingemo_dir, "scenes_5", f"scene_{idx}.jpg")
#         cv2.imwrite(save_path, blurred_image, [cv2.IMWRITE_JPEG_QUALITY, 90])

# save_scene_images()

In [ ]:
# def debug_query(model, pred_name, *args, substitution=None):
#     """Helper to test intermediate predicates"""
#     query = Query(Term(pred_name, *args), substitution=substitution)
#     print(f"\n{'='*60}")
#     print(f"Testing: {pred_name}")
#     print(f"{'='*60}")
#     model.eval()
#     results = model.solve([query])
#     result = results[0].result
    
#     # Print probabilities nicely
#     for key, prob in sorted(result.items(), key=lambda x: -x[1]):
#         print(f"  {key}: {prob:.4f}")
    
#     return result

In [ ]:
# faces_dir = os.path.join(findingemo_dir, "faces")
# os.makedirs(faces_dir, exist_ok=True)

# csv_path = os.path.join(findingemo_dir, "face_boxes.csv")

# # Create and open CSV file for writing
# with open(csv_path, "w", newline="") as f:
#     writer = csv.writer(f)
#     # header row
#     writer.writerow(["index", "image_path", "face_rank", "x1", "y1", "x2", "y2", "score", "crop_path"])

#     # ========================================
#     # Iterate over the dataframe and process each image
#     # ========================================
#     for idx, row in tqdm(df.iterrows(), total=len(df), desc="Detecting faces"):
#         img_path = os.path.join(findingemo_dir, row["image_path"])

#         try:
#             detections = RetinaFace.detect_faces(img_path)
#         except Exception as e:
#             print(f"[Warning] Could not process {img_path}: {e}")
#             continue

#         # Skip images without faces
#         if not detections or isinstance(detections, str):
#             continue

#         # Collect all detected faces
#         faces = []
#         for det in detections.values():
#             x1, y1, x2, y2 = det["facial_area"]
#             area = (x2 - x1) * (y2 - y1)
#             faces.append({
#                 "coords": (x1, y1, x2, y2),
#                 "score": det["score"],
#                 "area": area
#             })

#         # Sort faces by area (largest first) and keep top 3
#         faces = sorted(faces, key=lambda x: x["area"], reverse=True)[:3]

#         # Crop and save top faces
#         img = Image.open(img_path).convert("RGB")
#         for i, face in enumerate(faces):
#             x1, y1, x2, y2 = face["coords"]
#             crop = img.crop((x1, y1, x2, y2)).resize((224, 224))
#             crop_name = f"img_{idx}_face_{i}.jpg"
#             crop_path = os.path.join(faces_dir, crop_name)
#             crop.save(crop_path, "JPEG", quality=90)

#             # Write bounding box info to CSV
#             writer.writerow([
#                 idx,                      # image index
#                 row["image_path"],        # relative image path
#                 i,                        # face rank (0=largest)
#                 x1, y1, x2, y2,           # bounding box coordinates
#                 face["score"],            # detection confidence
#                 crop_path                 # saved crop file
#             ])

# print(f"Finished! Cropped faces saved in: {faces_dir}")
# print(f"Bounding box CSV saved at: {csv_path}")


In [ ]:
# def save_pretrained_logits_separately(indices, output_dir):
#     """
#     Save pre-computed logits from pretrained models to separate files.
    
#     Args:
#         indices: list of sample indices to process
#         output_dir: base directory to save the logits
#     """
#     # Create output directories
#     scene_dir = os.path.join(output_dir, "scenes")
#     faces_dir = os.path.join(output_dir, "faces")
#     os.makedirs(scene_dir, exist_ok=True)
#     os.makedirs(faces_dir, exist_ok=True)
    
#     # Set models to eval mode
#     face_model_base.eval()
#     scene_model_base.eval()
    
#     for img_idx in tqdm(indices, desc="Saving pretrained logits"):
#         # ----- PROCESS SCENE -----
#         scene_img = scenes_dataset[(img_idx,)].to(DEVICE).unsqueeze(0)  # (1, C, H, W)
        
#         with torch.no_grad():
#             scene_logits = scene_model_base(scene_img).squeeze(0)  # (365,)
        
#         # Save scene logits
#         scene_file = os.path.join(scene_dir, f"scene_{img_idx}.pt")
#         torch.save(scene_logits.cpu(), scene_file)
        
#         # ----- PROCESS FACES -----
#         face_rows = df_boxes[df_boxes['index'] == img_idx].sort_values('face_rank')
        
#         for _, row in face_rows.iterrows():
#             face_rank = int(row['face_rank'])
#             face_tensor = faces_dataset[(img_idx, face_rank)]
            
#             with torch.no_grad():
#                 face_tensor = face_tensor.to(DEVICE).unsqueeze(0)  # (1, C, H, W)
#                 outputs = face_model_base(face_tensor)  # Returns ImageClassifierOutput
#                 face_logits = outputs.logits.squeeze(0)  # (7,)
            
#             # Save face logits
#             face_file = os.path.join(faces_dir, f"face_{img_idx}_{face_rank}.pt")
#             torch.save(face_logits.cpu(), face_file)
    
#     print(f"\nLogits saved to:")
#     print(f"  Scenes: {scene_dir}")
#     print(f"  Faces: {faces_dir}")

# # Usage:
# # save_pretrained_logits_separately(indices, output_dir=os.path.join(data_dir, "pretrained_logits"))

In [ ]:
# def save_pretrained_logits_separately(indices, output_dir):
#     """
#     Save pre-computed logits from pretrained models to separate files.
    
#     Args:
#         indices: list of sample indices to process
#         output_dir: base directory to save the logits
#     """
#     # Create output directories
#     scene_dir = os.path.join(output_dir, "scenes")
#     os.makedirs(scene_dir, exist_ok=True)
    
#     # Set models to eval mode
#     scene_model_base.eval()
    
#     for img_idx in tqdm(indices, desc="Saving pretrained logits"):
#         # ----- PROCESS SCENE -----
#         scene_img = scenes_dataset[img_idx].to(DEVICE).unsqueeze(0)  # (1, C, H, W)
        
#         with torch.no_grad():
#             scene_logits = scene_model_base(scene_img)
#             scene_logits = scene_logits.view(scene_logits.size(0), -1)
        
#         # Save scene logits
#         scene_file = os.path.join(scene_dir, f"scene_{img_idx}.pt")
#         torch.save(scene_logits.cpu(), scene_file)
        
#     print(f"\nEmbeddings saved to:")
#     print(f"  Scenes: {scene_dir}")

# # Usage:
# save_pretrained_logits_separately(indices, output_dir=os.path.join(data_dir, "pretrained_backbone"))

In [ ]:
# # Show the image but with faces blurred out
# def blur_faces(image_path, boxes):
#     image = cv2.imread(image_path)
#     for box in boxes:
#         x1, y1, x2, y2 = box
#         face = image[y1:y2, x1:x2]
#         blurred_face = cv2.GaussianBlur(face, (99, 99), 30)
#         image[y1:y2, x1:x2] = blurred_face
#     return image

# img_path = os.path.join(findingemo_dir, df.iloc[9]['image_path'])
# boxes = df_boxes[df_boxes['index'] == 9][['x1', 'y1', 'x2', 'y2']].values.tolist()
# blurred_image = blur_faces(img_path, boxes)

# plt.imshow(cv2.cvtColor(blurred_image, cv2.COLOR_BGR2RGB))
# plt.axis('off')
# plt.show()

In [ ]:
# def show_image(dataframe, index):
#     img_path = os.path.join(findingemo_dir, dataframe.loc[index, 'image_path'])
#     print("Image path:", img_path)
#     image = cv2.imread(img_path)
#     image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
#     plt.imshow(image)
#     plt.axis('off')
#     plt.show()

In [ ]:
# def show_pretrained_model_predictions_on_sample(sample_idx):
#     row = df.iloc[sample_idx]
#     img_path = os.path.join(findingemo_dir, row['image_path'])
#     img = Image.open(img_path).convert("RGB")
    
#     # Convert to numpy array for drawing
#     img_array = np.array(img)
    
#     # Get all face boxes for this image
#     face_boxes = df_boxes[df_boxes['index'] == sample_idx].sort_values('face_rank')
    
#     # Draw bounding boxes and predictions on the image
#     for _, box_row in face_boxes.iterrows():
#         x1, y1, x2, y2 = int(box_row['x1']), int(box_row['y1']), int(box_row['x2']), int(box_row['y2'])
#         face_rank = int(box_row['face_rank'])
        
#         model_output = torch.load(os.path.join(data_dir, "pretrained_logits", "faces", f"face_{sample_idx}_{face_rank}.pt")).to(DEVICE).unsqueeze(0)
#         probs_output = torch.softmax(model_output, dim=1)
#         probs = probs_output[0].cpu().detach().numpy()
        
#         # Get top prediction
#         top_idx = np.argmax(probs)
#         top_label = f"{fer_classes[top_idx]}: {probs[top_idx]:.2f}"
        
#         # Draw rectangle
#         cv2.rectangle(img_array, (x1, y1), (x2, y2), (0, 255, 0), 2)
        
#         # Calculate text size for better positioning
#         font = cv2.FONT_HERSHEY_SIMPLEX
#         font_scale = 0.8
#         thickness = 2
#         (text_width, text_height), baseline = cv2.getTextSize(top_label, font, font_scale, thickness)
        
#         # Position text above the box if there's space, otherwise below
#         if y1 - text_height - 10 > 0:
#             text_y = y1 - 10
#         else:
#             text_y = y2 + text_height + 10
        
#         # Draw text with background for better visibility
#         cv2.rectangle(img_array, (x1, text_y - text_height - 5), 
#                     (x1 + text_width + 5, text_y + 5), (0, 255, 0), -1)
#         cv2.putText(img_array, top_label, (x1, text_y), 
#                     font, font_scale, (0, 0, 0), thickness)
        
#     # Display image with bounding boxes and predictions
#     plt.figure(figsize=(15, 10))
#     plt.imshow(img_array)
#     plt.axis('off')
#     plt.title(f"Image Index: {sample_idx}, Valence: {row['valence']}, Arousal: {row['arousal']}, Emotion: {row['emotion']}", fontsize=16)
#     plt.show()

#     scene_logits = torch.load(os.path.join(data_dir, "pretrained_logits", "scenes", f"scene_{sample_idx}.pt")).to(DEVICE)
#     scene_probs = torch.softmax(scene_logits, dim=0)
   

#     # Show top 3 from base model (365 categories)
#     top3_base = torch.topk(scene_probs, 3)
#     print("Base model (365 categories):")
#     for prob, idx in zip(top3_base.values.cpu(), top3_base.indices.cpu()):
#         print(f"  {get_category_name(idx)}: {prob:.4f}")
#     print("")


In [ ]:
# def calculate_places_priors(threshold = 0.10, indices = train_indices):
#     scene_model_base.eval()
#     scene_dict = {scene_name: [] for scene_name in scene_categories}
#     for sample_idx in tqdm(indices, desc=f"Calculating priors over {len(indices)} samples"):
#         scene_tensor = train_scenes_dataset[(sample_idx,)]
#         row = df.loc[sample_idx]
#         valence = row['valence'] + 3
#         arousal = row['arousal']
#         with torch.no_grad():
#             mapped_output = scene_model_base(scene_tensor.unsqueeze(0))[0]
#             mapped_softmax = torch.softmax(mapped_output, dim=0)

#         top3_scenes = torch.topk(mapped_softmax, 3)
        
#         # Now extract the scene names and probabilities
#         for prob, idx in zip(top3_scenes.values.cpu(), top3_scenes.indices.cpu()):
#             scene_name = scene_categories[idx.item()]
#             prob_value = prob.item()
#             if prob_value > threshold:
#                 scene_dict[scene_name].append((valence, arousal))

#     # Take the average of valence and arousal for each scene
#     scene_priors = {}
#     for scene_name, va_list in scene_dict.items():
#         if len(va_list) > 0:
#             nr_samples = len(va_list)
#             avg_valence = sum([va[0] for va in va_list]) / nr_samples
#             avg_arousal = sum([va[1] for va in va_list]) / nr_samples
#             scene_priors[scene_name] = (avg_valence, avg_arousal, nr_samples)
#         else:
#             scene_priors[scene_name] = (None, None, 0)

#     return scene_priors

# scene_priors = calculate_places_priors(threshold=0.10, indices=train_indices)

# # Save scene_priors to excel
# cluster_labels = []
# for scene_name in scene_categories:
#     valence, arousal, nr_samples = scene_priors[scene_name]
#     cluster_labels.append({
#         "scene_category": scene_name,
#         "valence": valence,
#         "arousal": arousal,
#         "nr_samples": nr_samples
#     })

# df_priors = pd.DataFrame(cluster_labels)

# output_path = os.path.join(data_dir, "scene_priors.xlsx")
# df_priors.to_excel(output_path, index=False)

# print(f"\nscene_priors.xlsx saved to: {output_path}")

In [ ]:
# def get_va_distribution(model, threshold=0.01):
#     emotion_va = {}
#     for emotion in fe_labels:
#         emotion_va[emotion] = {'valence': [], 'arousal': [], 'probs': []}

#     for train_idx in tqdm(train_indices):

#         output = debug_query(model,
#                 "test_combo",
#                 Term("tensor", Term("face_predictions", Constant(train_idx))),
#                 Term("tensor", Term("scene_predictions", Constant(train_idx))),
#                 Var("CA"),
#                 Var("CV"))
        
#         emotion = df.iloc[train_idx]['emotion']

#         for key, prob in output.items():
#             cv = float(key.args[2])
#             ca = float(key.args[3])
#             prob_val = float(prob)
            
#             if prob_val >= threshold:
#                 emotion_va[emotion]['valence'].append(cv)
#                 emotion_va[emotion]['arousal'].append(ca)
#                 emotion_va[emotion]['probs'].append(prob_val)  # Store probabilities

#     return emotion_va


# avg_emotion_va = {}
# for emotion, va_values in va_distribution.items():
#     if len(va_values['valence']) > 0:
#         # Convert to numpy arrays
#         valences = np.array(va_values['valence'])
#         arousals = np.array(va_values['arousal'])
#         probs = np.array(va_values['probs'])
        
#         # Normalize probabilities to sum to 1
#         probs_normalized = probs / probs.sum()
        
#         # Weighted average
#         avg_valence = np.sum(valences * probs_normalized)
#         avg_arousal = np.sum(arousals * probs_normalized)
#     else:
#         avg_valence = None
#         avg_arousal = None
    
#     avg_emotion_va[emotion] = (avg_valence, avg_arousal)

In [ ]:
# def closest_emotion(CV, CA):
#     # Extract values from Constant objects
#     cv_val = CV.value if isinstance(CV, Constant) else CV
#     ca_val = CA.value if isinstance(CA, Constant) else CA
    
#     closest_emo = None
#     min_distance = float('inf')
#     for emo, (EV, EA) in emotion_va_bins.items():
#         DX = cv_val - EV
#         DY = ca_val - EA
#         D = (DX * DX + DY * DY) ** 0.5
#         if D < min_distance:
#             min_distance = D
#             closest_emo = emo

#     emo_id = LABEL_TO_ID[closest_emo]

#     return Constant(emo_id)

In [ ]:
# model.register_foreign(closest_emotion, "closest_emotion", 2, 1)

In [ ]:
# combined_va_centroids = {'anger': (2.4171032456116555, 3.019359600458155),
#                         'anticipation': (2.8101052960354296, 2.90118067312196),
#                         'disgust': (2.561769099363139, 2.781531264549843),
#                         'fear': (2.4226634795792754, 2.7616159987215667),
#                         'joy': (3.8179193984605413, 3.4802071510276194),
#                         'sadness': (2.305686367051083, 2.63678745898695),
#                         'surprise': (2.9985175076110635, 3.067898037308043),
#                         'trust': (3.2087952323808873, 3.0928009251717663)}

In [ ]:
# class DeviceAwareModule(nn.Module):
#     """Wrapper that automatically handles device placement for any module"""
#     def __init__(self, module):
#         super().__init__()
#         self.module = module
    
#     def forward(self, *args, **kwargs):
#         # Move all tensor inputs to model's device
#         device = next(self.module.parameters()).device
        
#         def to_device(x):
#             if isinstance(x, torch.Tensor):
#                 return x.to(device)
#             elif isinstance(x, (list, tuple)):
#                 return type(x)(to_device(item) for item in x)
#             elif isinstance(x, dict):
#                 return {k: to_device(v) for k, v in x.items()}
#             return x
        
#         args = to_device(args)
#         kwargs = to_device(kwargs)
#         output = self.module(*args, **kwargs)

#         # Flatten output if it has extra dimensions
#         if isinstance(output, torch.Tensor) and output.dim() == 3:
#             output = output.squeeze(1)  # Remove the middle dimension [B, 1, C] -> [B, C]
        
#         return output
    
#     def __getattr__(self, name):
#         try:
#             return super().__getattr__(name)
#         except AttributeError:
#             return getattr(self.module, name)

In [ ]:
# class CombinationMLP(nn.Module):
#     def __init__(self, *sizes, activation=nn.ReLU, softmax=True, batch=True):
#         super(CombinationMLP, self).__init__()
#         layers = []
#         self.batch = batch
#         for i in range(len(sizes) - 2):
#             layers.append(nn.Linear(sizes[i], sizes[i + 1]))
#             layers.append(activation())
#         layers.append(nn.Linear(sizes[-2], sizes[-1]))
#         if softmax:
#             layers.append(nn.Softmax(-1))
#         self.nn = nn.Sequential(*layers)

#     def forward(self, face_features, scene_tensor):
#         x = torch.cat((face_features, scene_tensor), dim=-1)
#         if not self.batch:
#             x = x.unsqueeze(0)
#         x = self.nn(x)
#         return x

# positive_emotion_model_torch_raw = CombinationMLP(
#     372,
#     256, 128,
#     3,
#     activation=nn.ReLU,
#     softmax=True,
#     batch=True
# )

# positive_emotion_model_torch = DeviceAwareModule(positive_emotion_model_torch_raw).to(DEVICE)
# positive_emotion_network = Network(positive_emotion_model_torch, "positive_emotion_model", batching=True)
# positive_emotion_network.optimizer = optim.Adam(positive_emotion_model_torch.parameters(), lr=0.001)

In [ ]:
# class VAtoEmotionNet(nn.Module):
#     """Maps combined valence-arousal to emotion"""
#     def __init__(self, num_emotions=8):
#         super().__init__()
#         self.net = nn.Sequential(
#             nn.Linear(2, 64),
#             nn.ReLU(),
#             nn.Linear(64, 32),
#             nn.ReLU(),
#             nn.Linear(32, num_emotions),
#         )
    
#     def forward(self, valence, arousal):
#         # Convert inputs to tensors if they're lists/scalars
#         if not isinstance(valence, torch.Tensor):
#             valence = torch.tensor(valence, dtype=torch.float32)
#         if not isinstance(arousal, torch.Tensor):
#             arousal = torch.tensor(arousal, dtype=torch.float32)

#         # Ensure batch dimension for computation
#         if valence.dim() == 0:
#             valence = valence.unsqueeze(0)
#         if arousal.dim() == 0:
#             arousal = arousal.unsqueeze(0)

#         # Stack into (batch, 2)
#         x = torch.stack((valence, arousal), dim=-1)
#         logits = self.net(x)  # (batch, num_emotions)
#         probs = torch.softmax(logits, dim=-1)
        
#         # Squeeze batch dimension for single inputs
#         if probs.shape[0] == 1:
#             probs = probs.squeeze(0)  # (num_emotions,)
        
#         return probs

# # Create the network - set batching=False
# va_to_emotion_model = VAtoEmotionNet(num_emotions=len(fe_labels))
# va_to_emotion_network = Network(va_to_emotion_model, "va_to_emotion_model", batching=False)
# va_to_emotion_network.optimizer = optim.Adam(va_to_emotion_model.parameters(), lr=0.001)